In [ ]:
import numpy as np
from scipy.stats import norm
from numba import njit  

n1, n2 = 200, 200  # samples
X1, X2 = 160, 148  # successes

#  MLE of p1 and p2
p1_hat = X1 / n1
p2_hat = X2 / n2
psi_hat = p1_hat - p2_hat

#Fisher information matrix
I_p1 = n1 / (p1_hat * (1 - p1_hat))
I_p2 = n2 / (p2_hat * (1 - p2_hat))
fisher_info = np.array([[I_p1, 0],
                        [0, I_p2]])

# asymptotic standard error w/ delta method
se_psi = np.sqrt((p1_hat * (1 - p1_hat)) / n1 + (p2_hat * (1 - p2_hat)) / n2)

# 90% CI w/ delta method
z = norm.ppf(0.95)  # z-score 
margin_error = z * se_psi  # margin of error
ci_delta = (psi_hat - margin_error, psi_hat + margin_error)

# 90% CI w/ parametric bootstrap

@njit  # jitted function for bootstrap (ninja 4 speed)
def bootstrap_bootstrap(B, n1, p1, n2, p2):
    bootstrap_psi = np.empty(B)
    for b in range(B):
        X1_b = np.random.binomial(n1, p1)
        X2_b = np.random.binomial(n2, p2)
        bootstrap_psi[b] = (X1_b / n1) - (X2_b / n2)
    return bootstrap_psi

B = 1_000_000  # arbitrarily large
np.random.seed(243624)  # reproducibility
bootstrap_psi = bootstrap_bootstrap(B, n1, p1_hat, n2, p2_hat)
ci_bootstrap = np.percentile(bootstrap_psi, [5, 95])

# print results
print(f"MLE of psi: {psi_hat:.4f}")  # print psi_hat
print(f"90% CI using delta method: ({ci_delta[0]:.3f}, {ci_delta[1]:.3f})")
print(f"90% CI using bootstrap: ({ci_bootstrap[0]:.3f}, {ci_bootstrap[1]:.3f})")